In [25]:
import json
import requests
import urllib.request
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import fiona
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from shapely.geometry import Point
import mplleaflet

In [ ]:
response = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&orderby=time&limit=10000')

In [27]:
print(response.status_code)

200


In [45]:
live_eq = response.content

live_eq

b'{"type":"FeatureCollection","metadata":{"generated":1512702821000,"url":"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&minmagnitude=5.0&orderby=time&limit=10000","title":"USGS Earthquakes","status":200,"api":"1.5.8","limit":10000,"offset":1,"count":152},"features":[{"type":"Feature","properties":{"mag":6.2,"place":"74km NNE of L\'Esperance Rock, New Zealand","time":1512698999440,"updated":1512700347368,"tz":-720,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/us2000c1xc","detail":"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us2000c1xc&format=geojson","felt":null,"cdi":null,"mmi":5.25,"alert":"green","status":"reviewed","tsunami":0,"sig":591,"net":"us","code":"2000c1xc","ids":",us2000c1xc,","sources":",us,","types":",geoserve,losspager,origin,phase-data,shakemap,","nst":null,"dmin":18.436,"rms":1.42,"gap":73,"magType":"mwp","type":"earthquake","title":"M 6.2 - 74km NNE of L\'Esperance Rock, New Zealand"},"geometry":{"type":"Point","coordinates":

In [47]:
## API site: https://earthquake.usgs.gov/fdsnws/event/1/#format-geojson

url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&orderby=time&limit=10000'
request = requests.get(url)
b = bytes(request.content)
with fiona.BytesCollection(b) as f:
    crs = f.crs
    gdf = gpd.GeoDataFrame.from_features(f, crs=crs)
    print(gdf.head(20))

    alert  cdi      code                                             detail  \
0          0.0  72936601  https://earthquake.usgs.gov/fdsnws/event/1/que...   
1          0.0  17526374  https://earthquake.usgs.gov/fdsnws/event/1/que...   
2          3.4  2000c1xk  https://earthquake.usgs.gov/fdsnws/event/1/que...   
3          0.0  17526364  https://earthquake.usgs.gov/fdsnws/event/1/que...   
4          0.0  2000c1xg  https://earthquake.usgs.gov/fdsnws/event/1/que...   
5   green  0.0  2000c1xc  https://earthquake.usgs.gov/fdsnws/event/1/que...   
6          4.8  2000c1x9  https://earthquake.usgs.gov/fdsnws/event/1/que...   
7          0.0  17525067  https://earthquake.usgs.gov/fdsnws/event/1/que...   
8          0.0  17525064  https://earthquake.usgs.gov/fdsnws/event/1/que...   
9          0.0  72936591  https://earthquake.usgs.gov/fdsnws/event/1/que...   
10         0.0  72936586  https://earthquake.usgs.gov/fdsnws/event/1/que...   
11         0.0  37825119  https://earthquake.usgs.go

In [48]:
eq_df = gdf
eq_df.to_csv('/Users/Justin/Code/data-question-3-xtreme-united/eq_api_return.csv')

In [41]:
# pylab.rcParams['figure.figsize'] = 25, 25

# eq_df.plot()
# plt.show()

In [42]:
world = gpd.GeoDataFrame.from_file('/Users/Justin/Desktop/world.shp')

In [ ]:
pylab.rcParams['figure.figsize'] = 25, 25
base = world.plot(color='black', edgecolor='grey')

eq_df.plot(ax=base, marker='o', alpha = 0.3, column ='mag', cmap = 'autumn', markersize=7);

In [ ]:
plt.show()

In [35]:
top_eq = pd.DataFrame.from_csv('/Users/Justin/Code/data-question-3-xtreme-united/eq_df.csv')

In [36]:
top_eq.head()

,origin,country,lat,long,depth_km,magnitude,sec_effects,shaking_death,pde_total,utsu_total,em_total,other_deaths,deaths
1,1900-05-11 17:23,Japan,38.7,141.1,5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1900-07-12 06:25,Turkey,40.3,43.1,NaN,5.9,NaN,NaN,NaN,140.0,NaN,NaN,140.0
3,1900-10-29 09:11,Venezuela,11.0,-66.0,0,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1901-02-15 00:00,China,26.0,100.1,0,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1901-03-31 07:11,Bulgaria,43.4,28.7,NaN,6.4,NaN,NaN,NaN,4.0,NaN,NaN,4.0


In [37]:
geometry = [Point(xy) for xy in zip(top_eq.long, top_eq.lat)]
hist_eq_df = top_eq.drop(['long', 'lat'], axis=1)
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(hist_eq_df, crs=crs, geometry=geometry)

In [38]:
hist_eq_df.head()

,origin,country,depth_km,magnitude,sec_effects,shaking_death,pde_total,utsu_total,em_total,other_deaths,deaths,geometry
1,1900-05-11 17:23,Japan,5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (141.1 38.7)
2,1900-07-12 06:25,Turkey,NaN,5.9,NaN,NaN,NaN,140.0,NaN,NaN,140.0,POINT (43.1 40.3)
3,1900-10-29 09:11,Venezuela,0,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-66 11)
4,1901-02-15 00:00,China,0,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (100.1 26)
5,1901-03-31 07:11,Bulgaria,NaN,6.4,NaN,NaN,NaN,4.0,NaN,NaN,4.0,POINT (28.7 43.4)


In [39]:
pylab.rcParams['figure.figsize'] = 25, 25
base = world.plot(color='black', edgecolor='grey')

# eq_df.plot(ax=base, marker='o', column ='mag', cmap = 'autumn', markersize = 7);
hist_eq_df.plot(ax=base, marker='*', color = 'blue', markersize = 10);
ax.set_axis_off()
f.suptitle('Last 1000 earthquakes with magnitude > 6')

mplleaflet.show()

NameError: name 'ax' is not defined

In [ ]:
eq_df.plot(marker='o', column ='mag', cmap = 'autumn', markersize = 10);
# hist_eq_df.plot(marker='*', color = 'blue', markersize = 10);
# ax.set_axis_off()
# f.suptitle('Last 1000 earthquakes with magnitude > 6')

mplleaflet.show()